---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
#import matplotlib.pyplot as plt
#%matplotlib notebook

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [23]:
def graph_identification():
    #for g in P1_Graphs:
        #plt.figure(figsize=(7,8))
        #degrees = g.degree()
        #degree_values = sorted(set(degrees.values()))
        #histogram = [list(degrees.values()).count(i) / float(nx.number_of_nodes(g)) for i in degree_values]
        #plt.plot(degree_values, histogram,'o')
        #plt.xlabel('Degree')
        #plt.ylabel('Fraction of nodes')
        #plt.xscale('log')
        #plt.yscale('log')
        #plt.show()
        # Graph 0 and 3 are generated by PA
    #avgCls_Spath = []
    #for j in [1,2,4]:
        #g = P1_Graphs[j]
        #avgCls_Spath.append((nx.average_clustering(g),nx.average_shortest_path_length(g)))
    #print(avgCls_Spath)
    #avgCls_Spath = [(0.5642419635919628, 5.089871871871872), 
    #    (0.4018222222222227, 9.378702269692925), (0.0033037037037037037, 5.0785509568313305)]
    # Graph 1 and 2 are SW_L and Graph 4 is SW_H
    return ['PA','SW_L','SW_L','PA','SW_H']
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [24]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [25]:
#G.nodes(data=True)
#nx.get_node_attributes(G, 'ManagementSalary')
len(G.nodes())

1005

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [42]:
def salary_predictions():
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.svm import SVC
    from sklearn.neural_network import MLPClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import roc_auc_score
    ##
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Betweeness_centrality'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['Closeness_centrality'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['Degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['PR'] = pd.Series(nx.pagerank(G))
    #df['SPR'] = pd.Series(nx.pagerank(G,alpha=0.8))
    #scores = nx.hits(G)
    #df['Hub_score'] = pd.Series(scores[0])
    #df['Auth_score'] = pd.Series(scores[1])
    ##
    Train = df[~np.isnan(df['ManagementSalary'])]
    Test = df[np.isnan(df['ManagementSalary'])]
    features = list(df.columns)
    features.remove('ManagementSalary')
    X_train = Train[features]
    y_train = Train['ManagementSalary']
    X_test = Test[features]
    ##
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    #clf = SVC(gamma=1.0,C=15.0,probability=True).fit(X_train_scaled,y_train)
    #clf = MLPClassifier(hidden_layer_sizes = [100, 100], alpha = 5,random_state = 0, solver='lbfgs', verbose=0).fit(X_train_scaled,y_train)
    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train_scaled, y_train)
    #clf = LogisticRegression().fit(X_train_scaled,y_train)
    #clf = DecisionTreeClassifier(max_depth=5).fit(X_train_scaled, y_train)
    
    y_pred_proba = (clf.predict_proba(X_test_scaled)[:,1]).tolist()
    x_pred = clf.predict(X_train_scaled)
    print('roc_auc score on the training : ',roc_auc_score(x_pred,y_train))
    return pd.Series(y_pred_proba,X_test.index)
salary_predictions()
# Best classificier :  DecisionTree with max_depth = 5 or RandomForest
# ROC_AUC_score on the test set : 0.8804 for Dtree, and 0.955 for RandomForest

roc_auc score on the training :  1.0


1       0.002776
2       0.990000
5       1.000000
8       0.287634
14      0.021883
18      0.050814
27      0.102186
30      0.707399
31      0.313182
34      0.116159
37      0.092504
40      0.086464
45      0.003396
54      0.329618
55      0.637737
60      0.228380
62      1.000000
65      0.960000
77      0.020833
79      0.097405
97      0.001321
101     0.000000
103     0.370358
108     0.029082
113     0.063883
122     0.011111
141     0.286258
142     1.000000
144     0.010000
145     0.587193
          ...   
913     0.007987
914     0.000000
915     0.000000
918     0.106250
923     0.002000
926     0.033000
931     0.000000
934     0.000000
939     0.000000
944     0.000000
945     0.002000
947     0.010722
950     0.001845
951     0.000000
953     0.000000
959     0.000000
962     0.000000
963     0.126236
968     0.000222
969     0.001391
974     0.000222
984     0.020455
987     0.020000
989     0.000722
991     0.015000
992     0.000000
994     0.000000
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [14]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(5)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0


In [15]:
t = future_connections[np.isnan(future_connections['Future Connection'])]
t.head(5)

,Future Connection
"(107, 348)",NaN
"(542, 751)",NaN
"(20, 426)",NaN
"(50, 989)",NaN
"(942, 986)",NaN


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [16]:
def new_connections_predictions():
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    ## nodes community : 
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    ## features : 
    ## without community
    future_connections['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbours'] = [len(list(nx.common_neighbors(G, x[0], x[1]))) for x in future_connections.index]
    future_connections['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Resource Allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Adamic Adar'] = [i[2] for i in nx.adamic_adar_index(G, future_connections.index)]
    ## with community
    future_connections['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index)]   
    future_connections['ra_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, future_connections.index)]
    ## Train and test data
    Train = future_connections[~np.isnan(future_connections['Future Connection'])]
    Test = future_connections[np.isnan(future_connections['Future Connection'])]
    features = list(future_connections.columns)
    features.remove('Future Connection')
    X_train = Train[features]
    y_train = Train['Future Connection']
    X_test = Test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    ## predictions
    clf = RandomForestClassifier(n_estimators=100, max_depth=6).fit(X_train_scaled, y_train)
    y_pred_proba = (clf.predict_proba(X_test_scaled)[:,1]).tolist()
    
    #x_pred = clf.predict(X_train_scaled)
    #print('roc_auc score on the training : ',roc_auc_score(x_pred,y_train))
    
    return pd.Series(y_pred_proba,X_test.index)
new_connections_predictions()
# ROC_AUC_score on the test set = 0.9127

roc_auc score on the training :  0.933385841312


(107, 348)    0.025680
(542, 751)    0.012520
(20, 426)     0.575221
(50, 989)     0.012486
(942, 986)    0.012842
(324, 857)    0.012502
(13, 710)     0.120299
(19, 271)     0.149110
(319, 878)    0.012520
(659, 707)    0.012531
(49, 843)     0.012701
(208, 893)    0.012531
(377, 469)    0.013972
(405, 999)    0.015124
(129, 740)    0.012413
(292, 618)    0.017666
(239, 689)    0.012325
(359, 373)    0.015491
(53, 523)     0.392180
(276, 984)    0.012282
(202, 997)    0.012540
(604, 619)    0.041256
(270, 911)    0.012520
(261, 481)    0.061878
(200, 450)    0.992986
(213, 634)    0.012517
(644, 735)    0.034898
(346, 553)    0.012354
(521, 738)    0.011725
(422, 953)    0.013897
                ...   
(672, 848)    0.012520
(28, 127)     0.989404
(202, 661)    0.012277
(54, 195)     0.999991
(295, 864)    0.012847
(814, 936)    0.012396
(839, 874)    0.012842
(139, 843)    0.012502
(461, 544)    0.013077
(68, 487)     0.012541
(622, 932)    0.012604
(504, 936)    0.017753
(479, 528) 